# Introduction

This notebook serves as a guide for new users of the `chebai` package, which is used for working with chemical data, especially focusing on Gene Ontology (GO) and Swiss UniProt Protein data. This notebook will explain how to instantiate the main data class, how the data files are structured, and how to work with different molecule encodings.

One key aspect of the package is its **dataset management system**. In the training process, chemical datasets play a critical role by providing the necessary data for model learning and validation. The chebai package simplifies the handling of these datasets by **automatically creating** them as needed. This means that users do not have to manually prepare datasets before running models; the package will generate and organize the data files based on the parameters and encodings selected. This feature ensures that the right data is available and formatted properly.

---

# 1. Instantiation of a Data Class

To start working with `chebai`, you first need to instantiate a GO_UniProt data class. This class is responsible for managing, interacting with, and preprocessing the GO and UniProt data
### Inheritance Hierarchy

GO_UniProt data classes inherit from [`_DynamicDataset`](https://github.com/ChEB-AI/python-chebai/blob/dev/chebai/preprocessing/datasets/base.py#L597), which in turn inherits from [`XYBaseDataModule`](https://github.com/ChEB-AI/python-chebai/blob/dev/chebai/preprocessing/datasets/base.py#L22). Specifically:

- **`_DynamicDataset`**: This class serves as an intermediate base class that provides additional functionality or customization for datasets that require dynamic behavior. It inherits from `XYBaseDataModule`, which provides the core methods for data loading and processing.

- **`XYBaseDataModule`**: This is the base class for data modules, providing foundational properties and methods for handling and processing datasets, including data splitting, loading, and preprocessing.

In summary, GO_UniProt data classes are designed to manage and preprocess chemical data effectively by leveraging the capabilities provided by `XYBaseDataModule` through the `_DynamicDataset` intermediary.


### Configuration Parameters

Data classes related to proteins can be configured using the following main parameters:

- **`go_branch (str)`**: The Gene Ontology (GO) branch. The default value is `"all"`, which includes all branches of GO in the dataset.

- **`splits_file_path (str, optional)`**: Path to a CSV file containing data splits. If not provided, the class will handle splits internally. The default is `None`.

### Additional Input Parameters

To get more control over various aspects of data loading, processing, and splitting, you can refer to documentation of additional parameters in docstrings of the respective classes: [`_GOUniProtDataExtractor`](https://github.com/ChEB-AI/python-chebai/blob/dev/chebai/preprocessing/datasets/go_uniprot.py#L33), [`XYBaseDataModule`](https://github.com/ChEB-AI/python-chebai/blob/dev/chebai/preprocessing/datasets/base.py#L22), [`_DynamicDataset`](https://github.com/ChEB-AI/python-chebai/blob/dev/chebai/preprocessing/datasets/base.py#L597), etc.

### Available GOUniProt Data Classes

__Note__: Check the code implementation of classes [here](https://github.com/ChEB-AI/python-chebai/blob/dev/chebai/preprocessing/datasets/go_uniprot.py):

#### `GOUniProtOver250`

A class for extracting data from the Gene Ontology and Swiss UniProt dataset with a threshold of 250 for selecting classes.

- **Inheritance**: Inherits from `_GOUniProtOverX`.

#### `GOUniProtOver50`

A class for extracting data from the Gene Ontology and Swiss UniProt dataset with a threshold of 50 for selecting classes.

- **Inheritance**: Inherits from `_GOUniProtOverX`.


### Instantiation Example

In [12]:
from chebai.preprocessing.datasets.go_uniprot import GOUniProtOver250

In [13]:
go_class = GOUniProtOver250()

---

# 2. Preparation / Setup Methods

Once a GOUniProt data class instance is created, it typically requires preparation before use. This step is necessary to download or load the relevant data files and set up the internal data structures.
### Automatic Execution: 
These methods are executed automatically within the data class instance. Users do not need to call them explicitly, as the code internally manages the preparation and setup of data, ensuring that it is ready for subsequent use in training and validation processes.


### Why is Preparation Needed?

- **Data Availability**: The preparation step ensures that the required GOUniProt data files are downloaded or loaded, which are essential for analysis.
- **Data Integrity**: It ensures that the data files are transformed into a compatible format required for model input.

### Main Methods for Data Preprocessing

The data preprocessing in a data class involves two main methods:

1. **`prepare_data` Method**:
   - **Purpose**: This method checks for the presence of raw data in the specified directory. If the raw data is missing, it fetches the ontology, creates a dataframe, and saves it to a file (`data.pkl`). The dataframe includes columns such as IDs, data representations, and labels.
   - **Documentation**: [PyTorch Lightning - `prepare_data`](https://lightning.ai/docs/pytorch/stable/data/datamodule.html#prepare-data)

2. **`setup` Method**:
   - **Purpose**: This method sets up the data module for training, validation, and testing. It checks for the processed data and, if necessary, performs additional setup to ensure the data is ready for model input. It also handles cross-validation settings if enabled.
   - **Description**: Transforms `data.pkl` into a model input data format (`data.pt`), ensuring that the data is in a format compatible for input to the model. The transformed data contains the following keys: `ident`, `features`, `labels`, and `group`. This method uses a subclass of Data Reader to perform the transformation.
   - **Documentation**: [PyTorch Lightning - `setup`](https://lightning.ai/docs/pytorch/stable/data/datamodule.html#setup)

These methods ensure that the data is correctly prepared and set up for subsequent use in training and validation processes.

In [ ]:
go_class.prepare_data()
go_class.setup()

---

# 3. GOUniProt Data File Structure

1. **`Raw Data Files`**: (e.g., `.obo` file and `.dat` file)
   - **Description**: These files contain the raw GO ontology and Swiss UniProt data, which are downloaded directly from their respective websites. They serve as the foundation for data processing. Since there are no versions associated with this dataset, common raw files are used for all subsets of the data.
   - **File Paths**:
     - `data/GO_UniProt/raw/${filename}.obo`
     - `data/GO_UniProt/raw/${filename}.dat`

2. **`data.pkl`**
   - **Description**: This file is generated by the `prepare_data` method and contains the processed data in a dataframe format. It includes protein IDs, data representations (such as SMILES strings), and class columns with boolean values.
   - **File Path**: `data/GO_UniProt/${dataset_name}/processed/data.pkl`

3. **`data.pt`**
   - **Description**: Generated by the `setup` method, this file contains encoded data in a format compatible with the PyTorch library. It includes keys such as `ident`, `features`, `labels`, and `group`, making it ready for model input.
   - **File Path**: `data/GO_UniProt/${dataset_name}/processed/${reader_name}/data.pt`

4. **`classes.txt`**
   - **Description**: This file lists the selected GO or UniProt classes based on a specified threshold. It ensures that only the relevant classes are included in the dataset for analysis.
   - **File Path**: `data/GO_UniProt/${dataset_name}/processed/classes.txt`

5. **`splits.csv`**
   - **Description**: This file contains saved data splits from previous runs. During subsequent runs, it is used to reconstruct the train, validation, and test splits by filtering the encoded data (`data.pt`) based on the IDs stored in `splits.csv`.
   - **File Path**: `data/GO_UniProt/${dataset_name}/processed/splits.csv`

**Note**: If `go_branch` is specified, the `dataset_name` will include the branch name in the format `${dataset_name}_${go_branch}`. Otherwise, it will just be `${dataset_name}`.


---

# 4. Information Stored in the Files

## go-basic.obo

The `go-basic.obo` file is a key resource in the Gene Ontology (GO) dataset, containing the ontology data that defines various biological processes, molecular functions, and cellular components, as well as their relationships. This file is downloaded directly from the Gene Ontology Consortium and serves as the foundational raw data for further processing in GO-based applications.

### Structure of `go-basic.obo`

The `go-basic.obo` file is organized into blocks of text known as "term documents." Each block starts with a `[Term]` header and contains various attributes that describe a specific biological process, molecular function, or cellular component within the GO ontology. These attributes include identifiers, names, relationships to other terms, and more.

#### Example of a Term Document

```plaintext
[Term]
id: GO:0000032
name: cell wall mannoprotein biosynthetic process
namespace: biological_process
def: "The chemical reactions and pathways resulting in the formation of cell wall mannoproteins, any cell wall protein that contains covalently bound mannose residues." [GOC:ai]
synonym: "cell wall mannoprotein anabolism" EXACT []
is_a: GO:0006057 ! mannoprotein biosynthetic process
is_a: GO:0031506 ! cell wall glycoprotein biosynthetic process
```

### Breakdown of Attributes

Each term document in the `go-basic.obo` file consists of the following key attributes:

- **`[Term]`**: 
  - **Description**: Indicates the beginning of a new term in the ontology. Each term represents a distinct biological process, molecular function, or cellular component.

- **`id: GO:0000032`**: 
  - **Description**: A unique identifier for the biological term within the GO ontology.
  - **Example**: `GO:0000032` refers to the term "cell wall mannoprotein biosynthetic process."

- **`name: cell wall mannoprotein biosynthetic process`**: 
  - **Description**: The name of the biological process, molecular function, or cellular component being described.
  - **Example**: The name "cell wall mannoprotein biosynthetic process" is a descriptive label for the GO term with the identifier `GO:0000032`.

- **`namespace: biological_process`**: 
  - **Description**: Specifies which ontology the term belongs to. The main namespaces are `biological_process`, `molecular_function`, and `cellular_component`.

- **`is_a: GO:0006057`**: 
  - **Description**: Defines hierarchical relationships to other terms within the ontology. The `is_a` attribute indicates that the current term is a subclass or specific instance of the referenced term.
  - **Example**: The term `GO:0000032` ("cell wall mannoprotein biosynthetic process") is a subclass of `GO:0006057` and subclass of `GO:0031506`.


## uniprot_sprot.dat

The `uniprot_sprot.dat` file is a key component of the UniProtKB/Swiss-Prot dataset. It contains curated protein sequences with detailed annotation. Each entry in the file corresponds to a reviewed protein sequence, complete with metadata about its biological function, taxonomy, gene name, cross-references to other databases, and more. Below is a breakdown of the structure and key attributes in the file, using the provided example.


## Structure of `uniprot_sprot.dat`

The `uniprot_sprot.dat` file is organized into blocks of text, each representing a single protein entry. These blocks contain specific tags and fields that describe different aspects of the protein, including its sequence, function, taxonomy, and cross-references to external databases.

### Example of a Protein Entry

```plaintext
ID   002L_FRG3G              Reviewed;         320 AA.
AC   Q6GZX3;
DT   28-JUN-2011, integrated into UniProtKB/Swiss-Prot.
DT   19-JUL-2004, sequence version 1.
DT   08-NOV-2023, entry version 46.
DE   RecName: Full=Uncharacterized protein 002L;
GN   ORFNames=FV3-002L;
OS   Frog virus 3 (isolate Goorha) (FV-3).
OC   Viruses; Varidnaviria; Bamfordvirae; Nucleocytoviricota; Megaviricetes;
OX   NCBI_TaxID=654924;
OH   NCBI_TaxID=8404; Lithobates pipiens (Northern leopard frog) (Rana pipiens).
RN   [1]
RP   NUCLEOTIDE SEQUENCE [LARGE SCALE GENOMIC DNA].
RX   PubMed=15165820; DOI=10.1016/j.virol.2004.02.019;
RA   Tan W.G., Barkman T.J., Gregory Chinchar V., Essani K.;
RT   "Comparative genomic analyses of frog virus 3, type species of the genus
RT   Ranavirus (family Iridoviridae).";
RL   Virology 323:70-84(2004).
CC   -!- SUBCELLULAR LOCATION: Host membrane {ECO:0000305}; Single-pass membrane
CC       protein {ECO:0000305}.
DR   EMBL; AY548484; AAT09661.1; -; Genomic_DNA.
DR   RefSeq; YP_031580.1; NC_005946.1.
DR   GeneID; 2947774; -.
DR   KEGG; vg:2947774; -.
DR   Proteomes; UP000008770; Segment.
DR   GO; GO:0033644; C:host cell membrane; IEA:UniProtKB-SubCell.
DR   GO; GO:0016020; C:membrane; IEA:UniProtKB-KW.
PE   4: Predicted;
KW   Host membrane; Membrane; Reference proteome; Transmembrane;
KW   Transmembrane helix.
FT   CHAIN           1..320
FT                   /note="Uncharacterized protein 002L"
FT                   /id="PRO_0000410509"
SQ   SEQUENCE   320 AA;  34642 MW;  9E110808B6E328E0 CRC64;
     MSIIGATRLQ NDKSDTYSAG PCYAGGCSAF TPRGTCGKDW DLGEQTCASG FCTSQPLCAR
     IKKTQVCGLR YSSKGKDPLV SAEWDSRGAP YVRCTYDADL IDTQAQVDQF VSMFGESPSL
     AERYCMRGVK NTAGELVSRV SSDADPAGGW CRKWYSAHRG PDQDAALGSF CIKNPGAADC
     KCINRASDPV YQKVKTLHAY PDQCWYVPCA ADVGELKMGT QRDTPTNCPT QVCQIVFNML
     DDGSVTMDDV KNTINCDFSK YVPPPPPPKP TPPTPPTPPT PPTPPTPPTP PTPRPVHNRK
     VMFFVAGAVL VAILISTVRW
//
```

### Breakdown of Attributes

Each protein entry in the `uniprot_sprot.dat` file is structured with specific tags and sections that describe the protein in detail. Here's a breakdown of the key attributes:

- **`ID`**: 
  - **Description**: Contains the unique identifier for the protein and its status (e.g., `Reviewed` indicates the sequence has been manually curated).
  - **Example**: `002L_FRG3G` is the identifier for the protein from Frog virus 3.

- **`AC`**: 
  - **Description**: Accession number, a unique identifier for the protein sequence.
  - **Example**: `Q6GZX3` is the accession number for this entry.

- **`DR`**: 
  - **Description**: Cross-references to other databases like EMBL, RefSeq, KEGG, and GeneID.
  - **Example**: This entry is cross-referenced with the EMBL database, RefSeq, GO, etc.

- **`GO`**: 
  - **Description**: Gene Ontology annotations that describe the cellular component, biological process, or molecular function associated with the protein.
  - **Example**: The protein is associated with the GO terms `GO:0033644` (host cell membrane) and `GO:0016020` (membrane).

- **`SQ`**: 
  - **Description**: The amino acid sequence of the protein.
  - **Example**: The sequence consists of 320 amino acids.

The `uniprot_sprot.dat` file is an extensively curated resource, containing comprehensive protein data used for various bioinformatics applications.

__Note__: For more detailed information refer [here](https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/docs/keywlist.txt
). 

Consider the below line from above example: 
```plaintext
DR   GO; GO:0033644; C:host cell membrane; IEA:UniProtKB-SubCell.
```

The line contains a **Gene Ontology (GO) annotation** describing the protein's subcellular location. Here's a detailed breakdown:

- **`GO:0033644`**: This is the specific **GO term** identifier for "host cell membrane," which indicates that the protein is associated with or located at the membrane of the host cell.

- **`IEA`**: This stands for **Inferred from Electronic Annotation**, which is part of the **GO Evidence Codes**. **IEA** indicates that the annotation was automatically generated based on computational methods rather than direct experimental evidence. While **IEA** annotations are useful, they are generally considered less reliable than manually curated or experimentally verified evidence codes.



## data.pkl

The `data.pkl` file, generated during the preprocessing stage, contains the processed GO data in a dataframe format. Below is an example of how this data is structured:



### Structure of `data.pkl`
`data.pkl` as following structure: 
- **Column 0**: Contains the Identifier from Swiss-UniProt Dataset for each Swiss Protein data instance.
- **Column 1**: Contains the accession of each Protein data instance.
- **Column 2**: Contains the list of GO-IDs (Identifiers from Gene Ontology) which maps each Swiss Protein to the Gene Ontology instance.
- **Column 3**: Contains the sequence representation for the Swiss Protein using Amino Acid notation.
- **Column 4 and onwards**: Contains the labels, starting from column 4.

This structure ensures that the data is organized and ready for further processing, such as further encoding.


In [3]:
import pandas as pd

In [7]:
pkl_df = pd.DataFrame(pd.read_pickle(r"data/GO_UniProt/GO250_BP/processed/data.pkl"))
print("Size of the data (rows x columns): ", pkl_df.shape)
pkl_df.head()

Size of the data (rows x columns):  (27459, 1050)


,swiss_id,accession,go_ids,sequence,41,75,122,165,209,226,...,2000145,2000146,2000147,2000241,2000243,2000377,2001020,2001141,2001233,2001234
8,14331_ARATH,"P42643,Q945M2,Q9M0S7",[19222],MATPGASSARDEFVYMAKLAEQAERYEEMVEFMEKVAKAVDKDELT...,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,14331_CAEEL,"P41932,Q21537","[132, 1708, 5634, 5737, 5938, 6611, 7346, 8340...",MSDTVEELVQRAKLAEQAERYDDMAAAMKKVTEQGQELSNEERNLL...,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10,14331_MAIZE,P49106,"[3677, 5634, 10468, 44877]",MASAELSREENVYMAKLAEQAERYEEMVEFMEKVAKTVDSEELTVE...,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
13,14332_MAIZE,Q01526,"[3677, 5634, 10468, 44877]",MASAELSREENVYMAKLAEQAERYEEMVEFMEKVAKTVDSEELTVE...,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14,14333_ARATH,"P42644,F4KBI7,Q945L2","[5634, 5737, 6995, 9409, 9631, 16036, 19222, 5...",MSTREENVYMAKLAEQAERYEEMVEFMEKVAKTVDVEELSVEERNL...,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## data.pt

The `data.pt` file is a list where each element is a dictionary with the following keys:

- **`features`**: 
  - **Description**: This key holds the input features for the model. The features are typically stored as tensors and represent the attributes used by the model for training and evaluation.

- **`labels`**: 
  - **Description**: This key contains the labels or target values associated with each instance. Labels are also stored as tensors and are used by the model to learn and make predictions.

- **`ident`**: 
  - **Description**: This key holds identifiers for each data instance. These identifiers help track and reference the individual samples in the dataset.


In [8]:
import torch

In [11]:
data_pt = torch.load(r"data/GO_UniProt/GO250_BP/processed/protein_token/data.pt")
print("Type of loaded data:", type(data_pt))
for i in range(1):
    print(data_pt[i])

Type of loaded data: <class 'list'>
{'features': [10, 14, 15, 23, 13, 14, 11, 11, 14, 16, 20, 27, 25, 28, 22, 10, 14, 21, 17, 14, 27, 18, 14, 27, 16, 22, 27, 27, 10, 28, 27, 25, 10, 27, 21, 28, 14, 21, 14, 28, 20, 21, 20, 27, 17, 15, 28, 27, 27, 16, 19, 17, 17, 11, 28, 14, 22, 21, 19, 28, 12, 13, 14, 16, 16, 14, 11, 26, 16, 12, 12, 11, 11, 12, 27, 18, 21, 27, 27, 11, 16, 13, 19, 20, 20, 29, 28, 11, 17, 12, 16, 20, 22, 16, 11, 21, 12, 27, 15, 27, 17, 11, 20, 12, 24, 20, 13, 12, 17, 21, 17, 17, 20, 15, 12, 17, 28, 23, 14, 14, 14, 11, 13, 20, 11, 21, 28, 25, 22, 17, 21, 10, 21, 13, 20, 22, 29, 16, 22, 17, 14, 27, 25, 21, 11, 13, 18, 27, 16, 21, 20, 14, 14, 27, 29, 15, 17, 15, 14, 22, 21, 14, 14, 18, 20, 12, 14, 19, 11, 27, 17, 14, 23, 15, 29, 23, 12, 16, 17, 13, 17, 14, 17, 19, 25, 11, 28, 25, 22, 22, 27, 12, 17, 19, 11, 23, 20, 16, 14, 24, 19, 17, 14, 21, 18, 14, 25, 20, 27, 14, 12, 14, 27, 17, 20, 15, 17, 13, 27, 27, 11, 22, 21, 20, 11, 15, 17, 12, 10, 18, 17, 17, 16, 20, 19, 17, 15, 17

## `classes.txt` File

The `classes.txt` file lists selected Swiss Proteins classes. These classes are chosen based on a specified threshold, which is typically used for filtering or categorizing the dataset. Each line in the file corresponds to a unique Swiss Protein class ID, identifying specific protein from Swiss-UniProt dataset.

This file is essential for organizing the data and ensuring that only relevant classes, as defined by the threshold, are included in subsequent processing and analysis tasks.


In [15]:
with open(r"data/GO_UniProt/GO250_BP/processed/classes.txt", "r") as file:
    for i in range(5):
        line = file.readline()
        print(line.strip())

41
75
122
165
209


## `splits.csv` File

The `splits.csv` file contains the saved data splits from previous runs, including the train, validation, and test sets. During subsequent runs, this file is used to reconstruct these splits by filtering the encoded data (`data.pt`) based on the IDs stored in `splits.csv`. This ensures consistency and reproducibility in data splitting, allowing for reliable evaluation and comparison of model performance across different run.

In [16]:
csv_df = pd.read_csv(r"data/GO_UniProt/GO250_BP/processed/splits.csv")
csv_df.head()

,id,split
0,14331_ARATH,train
1,14331_CAEEL,train
2,14331_MAIZE,train
3,14332_MAIZE,train
4,14333_ARATH,train


---

## Protein Representation Using Amino Acid Sequence Notation

Proteins are composed of chains of amino acids, and these sequences can be represented using a one-letter notation for each amino acid. This notation provides a concise way to describe the primary structure of a protein.

### Example Protein Sequence

Protein: **Lysozyme C** from **Gallus gallus** (Chicken).  
[Lysozyme C - UniProtKB P00698](https://www.uniprot.org/uniprotkb/P00698/entry#function)

- **Sequence**: `MRSLLILVLCFLPLAALGKVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRNTDGSTDYGILQINSRWWCNDGRTPGSRNLCNIPCSALLSSDITASVNCAKKIVSDGNGMNAWVAWRNRCKGTDVQAWIRGCRL`
- **Sequence Length**: 147

In this sequence, each letter corresponds to a specific amino acid. This notation is widely used in bioinformatics and molecular biology to represent protein sequences.

### The 20 Amino Acids and Their One-Letter Notations

Here is a list of the 20 standard amino acids, along with their one-letter notations and descriptions:

| One-Letter Notation | Amino Acid Name      | Description                                             |
|---------------------|----------------------|---------------------------------------------------------|
| **A**               | Alanine              | Non-polar, aliphatic amino acid.                        |
| **C**               | Cysteine             | Polar, contains a thiol group, forms disulfide bonds.   |
| **D**               | Aspartic Acid        | Acidic, negatively charged at physiological pH.         |
| **E**               | Glutamic Acid        | Acidic, negatively charged at physiological pH.         |
| **F**               | Phenylalanine        | Aromatic, non-polar.                                    |
| **G**               | Glycine              | Smallest amino acid, non-polar.                         |
| **H**               | Histidine            | Polar, positively charged, can participate in enzyme active sites. |
| **I**               | Isoleucine           | Non-polar, aliphatic.                                   |
| **K**               | Lysine               | Basic, positively charged at physiological pH.          |
| **L**               | Leucine              | Non-polar, aliphatic.                                   |
| **M**               | Methionine           | Non-polar, contains sulfur, start codon in mRNA translation. |
| **N**               | Asparagine           | Polar, uncharged.                                       |
| **P**               | Proline              | Non-polar, introduces kinks in protein chains.          |
| **Q**               | Glutamine            | Polar, uncharged.                                       |
| **R**               | Arginine             | Basic, positively charged, involved in binding phosphate groups. |
| **S**               | Serine               | Polar, can be phosphorylated.                           |
| **T**               | Threonine            | Polar, can be phosphorylated.                           |
| **V**               | Valine               | Non-polar, aliphatic.                                   |
| **W**               | Tryptophan           | Aromatic, non-polar, largest amino acid.                |
| **Y**               | Tyrosine             | Aromatic, polar, can be phosphorylated.                 |

### Understanding Protein Sequences

In the example sequence, each letter represents one of the above amino acids. The sequence reflects the specific order of amino acids in the protein, which is critical for its structure and function.

This notation is used extensively in various bioinformatics tools and databases to study protein structure, function, and interactions.


_Note_:  Refer for amino acid sequence:  https://en.wikipedia.org/wiki/Protein_primary_structure

---

## More on GO Evidence Codes

The **Gene Ontology (GO) Evidence Codes** provide a way to indicate the level of evidence supporting a GO annotation. Here's a list of the both **experimental** and **non-experimental** GO evidence codes with brief descriptions:

| **Evidence Code** | **Description** |
|-------------------|-----------------|
| **EXP**           | Inferred from Experiment |
| **IDA**           | Inferred from Direct Assay |
| **IPI**           | Inferred from Physical Interaction |
| **IMP**           | Inferred from Mutant Phenotype |
| **IGI**           | Inferred from Genetic Interaction |
| **IEP**           | Inferred from Expression Pattern |
| **TAS**           | Traceable Author Statement |
| **IC**            | Inferred by Curator |
| **IEA**           | Inferred from Electronic Annotation (Computational) |
| **ISS**           | Inferred from Sequence or Structural Similarity |
| **ISA**           | Inferred from Sequence Alignment |
| **ISM**           | Inferred from Sequence Model |
| **ISO**           | Inferred from Sequence Orthology |
| **ISA**           | Inferred from Sequence Alignment |
| **RCA**           | Inferred from Reviewed Computational Analysis |
| **NAS**           | Non-traceable Author Statement |
| **ND**            | No Biological Data Available (placeholder) |
| **NR**            | Not Recorded |


### Grouping of Codes:

- **Experimental Evidence Codes**: 
  - **EXP**, **IDA**, **IPI**, **IMP**, **IGI**, **IEP**
  
- **Author/Curator Inferred Codes**:
  - **TAS**, **IC**, **NAS**

- **Computational Evidence Codes**:
  - **IEA**, **ISS**, **ISA**, **ISM**, **ISO**, **RCA**

- **Others**:
  - **ND** (No Data), **NR** (Not Recorded)


These evidence codes ensure transparency and give researchers an understanding of how confident they can be in a particular GO annotation.

---